In [234]:
import pandas as pd

In [64]:
def team_rank(year):
    dfs = pd.read_html(f'https://www.basketball-reference.com/playoffs/NBA_{year}.html')
    df = dfs[20]  # the third table in the page and remove the last row 
    df.columns = df.columns.droplevel()
    if year==2023:
        df = df.loc[:,['Team','W','L']]
    else:
        df = df.loc[:,['Tm','W','L']]
        df['Team']=df['Tm']
        df = df.drop('Tm',axis=1)
        df.drop(16,inplace=True)
    if year>=2003:
        df["Round Finished"] = df.apply(lambda row: 5 if row["W"] == 16 else
                                 4 if row["W"] >= 12 else
                                 3 if 8 <= row["W"] < 12 else
                                 2 if 4 <= row["W"] < 8 else
                                 1 if row["W"] < 4 and row["L"]>0 else
                                 -1, axis=1)
    else:
        df["Round Finished"] = df.apply(lambda row: 5 if row["W"] == 15 else
                                 4 if row["W"] >= 11 else
                                 3 if 7 <= row["W"] < 11 else
                                 2 if 3 <= row["W"] < 7 else
                                 1 if row["W"] < 3 and row["L"]>0 else
                                 -1, axis=1)          
    df = df.drop(['W','L'],axis=1)
    new = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_standings.html')
    east = new[0]
    west = new[1]
    west = west["Western Conference"].apply(lambda x: x.split("*")[0].strip()).str.split("(", expand=True)[0].str.strip()
    east = east["Eastern Conference"].apply(lambda x: x.split("*")[0].strip()).str.split("(", expand=True)[0].str.strip()
    teams = list(pd.concat([east,west]).reset_index(drop=True))
    for value in teams:
        if value not in df['Team'].values:
            df.loc[len(df)] = { 'Team': value, 'Round Finished': -1}
    df = df.sort_values('Round Finished',ascending = False).reset_index(drop = True)
    return df


In [67]:
empty_df = pd.DataFrame()
for i in range(1999,2024,1):
    print(i)
    df = team_rank(i)
    df['Year']=i
    result = pd.concat([empty_df,df], axis=0) 

1999


HTTPError: HTTP Error 429: Too Many Requests